In this notebook we will test the revision module 

In [1]:
## IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.revision import revision
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [3]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the candidate SQL 
filepath_query = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/generated_candidate.json"
generated_candidate = load_json_data(filepath_query)

In [4]:
# test candidate generation module with a random sample
index = 9
example = data[index]
task = create_task(example)
model = "gpt-4"
ans = revision(task=task, retrieved_entities=retrieved_entities[index],
               retrieved_context=retrieved_context[index], generated_candidate=generated_candidate[index],
               model=model,
               num_samples=1)
ans

{'chain_of_thought_reasoning': "To find the potential of Francesco Parravicini on a specific date, I joined the Player and Player_Attributes tables using the player_api_id as the common key. I filtered the results for the player name 'Francesco Parravicini' and the specific date '2010-08-30 00:00:00'. I also ensured that the potential column is not null to avoid any null results. The query strictly adheres to the database admin instructions, particularly focusing on joining tables with aliases, filtering null values, and selecting only the required column.",
 'SQL': "SELECT T2.potential FROM Player T1 JOIN Player_Attributes T2 ON T1.player_api_id = T2.player_api_id WHERE T1.player_name = 'Francesco Parravicini' AND T2.date = '2010-08-30 00:00:00' AND T2.potential IS NOT NULL"}

In [5]:
#test revision module with the subsampled test
model = "gpt-4"
num_samples = 1
res = []
for index, sample in tqdm(enumerate(data)):
    task = create_task(sample)
    ans = revision(task=task, retrieved_entities=retrieved_entities[index],
                   retrieved_context=retrieved_context[index], generated_candidate=generated_candidate[index],
                   model=model,
                   num_samples=num_samples)
    res.append(ans)
    print(str(index) + " : ", ans)

1it [00:09,  9.19s/it]

0 :  {'chain_of_thought_reasoning': "The query aims to list each atom ID from the 'atom' table that belongs to the molecule 'TR346' and count the distinct types of bonds that can be formed by this molecule, as indicated by the 'bond' table. The JOIN operation is used to combine rows from 'atom' and 'bond' tables based on the molecule_id, ensuring that only entries where molecule_id equals 'TR346' are considered. The COUNT(DISTINCT) function is used to ensure that each type of bond is counted only once per atom, regardless of how many times it appears. The GROUP BY clause groups the results by atom_id, ensuring that the count of bond types is calculated for each atom individually.", 'SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) FROM atom T1 JOIN bond T2 ON T1.molecule_id = T2.molecule_id WHERE T1.molecule_id = 'TR346' AND T2.molecule_id = 'TR346' GROUP BY T1.atom_id"}


2it [00:22, 11.90s/it]

1 :  {'chain_of_thought_reasoning': "To find the city location of the high school with specific criteria, I joined the 'schools' and 'frpm' tables using the CDSCode as a common key. I filtered the 'schools' table for schools in the county of Merced and for high schools, which are indicated by the EILCode 'HS'. In the 'frpm' table, I filtered for schools with 'Lunch Provision 2' as the NSLP Provision Status and with the lowest grade as 9 and the highest grade as 12. The query then selects the city from the 'schools' table that meets all these conditions.", 'SQL': "SELECT T1.City FROM schools T1 JOIN frpm T2 ON T1.CDSCode = T2.CDSCode WHERE T1.County = 'Merced' AND T1.EILCode = 'HS' AND T2.`NSLP Provision Status` = 'Lunch Provision 2' AND T2.`Low Grade` = '9' AND T2.`High Grade` = '12'"}


3it [00:31, 10.14s/it]

...

12 :  {'chain_of_thought_reasoning': "The task is to calculate the percentage of bonds that are double bonds ('=') among all bonds in molecule TR047. The SQL query uses a CASE statement within the SUM function to count only the double bonds, and divides this by the total count of bonds for molecule TR047, multiplying by 100 to get the percentage. The WHERE clause ensures that only bonds from molecule TR047 are considered. This approach adheres to the database admin instructions by directly calculating the percentage in a single query without subqueries, and by not including unnecessary columns in the SELECT clause.", 'SQL': "SELECT (SUM(CASE WHEN bond_type = '=' THEN 1 ELSE 0 END) * 100.0 / COUNT(bond_id)) AS percent FROM bond WHERE molecule_id = 'TR047'"}


14it [02:40, 11.21s/it]

13 :  {'chain_of_thought_reasoning': "To find the user who added a bounty amount of 50 to a post with a title mentioning 'variance', I joined three tables: votes, posts, and users. The votes table (T1) connects to the posts table (T2) via PostId to access the post details, and to the users table (T3) via UserId to get the user's DisplayName. I filtered the votes for those with a BountyAmount of 50 and posts whose titles contain 'variance'. I also ensured that BountyAmount and Title are not null to avoid errors in filtering and matching conditions.", 'SQL': "SELECT T3.DisplayName FROM votes T1 JOIN posts T2 ON T1.PostId = T2.Id JOIN users T3 ON T1.UserId = T3.Id WHERE T1.BountyAmount = 50 AND T2.Title LIKE '%variance%' AND T1.BountyAmount IS NOT NULL AND T2.Title IS NOT NULL"}


15it [02:48, 11.24s/it]

14 :  {'chain_of_thought_reasoning': "The query aims to retrieve the dates when users from 'Rochester, NY' obtained their badges. The users table is joined with the badges table on the user ID to access the badge dates. The WHERE clause filters users based on the location 'Rochester, NY' and ensures that the badge date is not null, adhering to the database admin instructions to filter out null values in logical operations.", 'SQL': "SELECT T2.Date FROM users T1 JOIN badges T2 ON T1.Id = T2.UserId WHERE T1.Location = 'Rochester, NY' AND T2.Date IS NOT NULL"}


In [6]:
from pathlib import Path

schema_path = Path('C:/Users/yousf/Bureau/ConvergenceAI/CHESS_Impl/data/test/final_sql.json')
with open(schema_path, 'w') as f:
    json.dump(res, f, indent=4)

## Cost Estimation per task

In [7]:
PROMPT_PATH = os.getenv("PROMPT_ROOT_PATH") + "\\candidate_generation.txt"
prompt = load_prompt(PROMPT_PATH)

In [8]:
def tokens_calc(example):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(example))
    return num_tokens

In [9]:
#prompt template tokens 
tokens_calc(prompt)

727

The prompt template has 727 tokens in total, but it have so many variables so it will more longer than
that to estimate the approximate token numbers we will see with a formatted prompt.
Let's say approximately 2200 tokens


In [10]:
## Output tokens estimation from an example 
output_example = """
{'chain_of_thought_reasoning': "To solve the query, I first identified the tables and columns relevant to the question. The question asks for the full name and college of a club member with the position of 'Secretary'. The 'member' table contains member details including position, and the 'major' table contains college information. The 'position' column in the 'member' table is used to filter for 'Secretary'. The 'link_to_major' in the 'member' table and 'major_id' in the 'major' table are used to join these tables. The 'college' column in the 'major' table provides the college information. I ensured to select only the columns explicitly asked for in the question, adhering to the database admin instructions.",
 'SQL': "SELECT T1.first_name, T1.last_name, T2.college FROM member AS T1 JOIN major AS T2 ON T1.link_to_major = T2.major_id WHERE T1.position = 'Secretary' AND T1.first_name IS NOT NULL AND T1.last_name IS NOT NULL AND T2.college IS NOT NULL"}
"""
tokens_calc(output_example)

228

Let's suppose the number of output tokens is 250

In [13]:
## Price calculation (just with gpt4 because in this module we don't use gpt3.5) 
input_price_per_token_gpt4 = 0.01 / 1000
output_price_per_token_gpt4 = 0.03 / 1000
price_gpt4 = 2200 * input_price_per_token_gpt4 + 250 * output_price_per_token_gpt4
print("estimated price per Task (GPT-4):", price_gpt4, "$")

estimated price per Task (GPT-4): 0.029500000000000002 $


In [14]:
## in this module there is a number of retrials so let's estimate the price with different number of retrials 

num_retrials = [1, 2, 3, 4, 5]
for num_retrial in num_retrials:
    total_price = price_gpt4 * num_retrial
    print("The estimated price with number of retrials of " + str(num_retrial) + ": ", total_price)
    

The estimated price with number of retrials of 1:  0.029500000000000002
The estimated price with number of retrials of 2:  0.059000000000000004
The estimated price with number of retrials of 3:  0.08850000000000001
The estimated price with number of retrials of 4:  0.11800000000000001
The estimated price with number of retrials of 5:  0.14750000000000002
